# Setup

In [1]:
# !pip install 'crewai[tools]' -q

In [ ]:
# Retrieve the API keys securely from Google Colab's user data
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api')
# serper_api_key = userdata.get('serper_api')



In [33]:
import os
import dotenv 

# Load environment variables from a .env file
dotenv.load_dotenv()
# Set the environment variables for OpenAI and Serper API keys

os.environ['GEMINI_API_KEY'] = os.environ.get('GEMINI_API_KEY')
os.environ['SERPER_API_KEY'] = os.environ.get('SERPER_API_KEY')
os.environ['GOOGLE_API_KEY'] = os.environ.get('GOOGLE_API_KEY')

In [17]:
from crewai import Agent, Crew, Process, Task, LLM
# from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, YoutubeChannelSearchTool, YoutubeVideoSearchTool

In [42]:
# # Define the LLM
# llm = ChatOpenAI(model_name='gpt-4o', temperature=0.5)

llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.5,   
)

llm.call("Helo what model are you")

'I am a large language model, trained by Google.\n'

# Data Inputs

In [10]:
brand_info = {
    "name": "Zero to Mastery",
    "description": "An online tech academy that offers comprehensive courses and a community for learners seeking in-demand software and tech skills.",
    "brand_values": "Quality education, community-driven growth, career-focused learning, and up-to-date tech stacks.",
    "ideal_partner": "A YouTube channel focused on programming, software engineering, or tech career advice with an audience of motivated learners."
}

In [11]:
# Define a single YouTube lead for demonstration
youtube_lead = {
    "name": "Theo - t3․gg",
    "channel_url": "https://www.youtube.com/@t3dotgg"
}

# Research

In [45]:
research_agent = Agent(
    role = "Influencer researcher",
    goal= f"Research about {youtube_lead['name']} and their YouTube channel {youtube_lead['channel_url']} to assess if they are a good fit for a partnership with {brand_info['name']}.",
    backstory=f"You are an expert in Influencer marketing and partnerships. You have a deep understanding of the tech education space and know how to evaluate potential partners based on their audience, content, and engagement.",
    llm=llm,
    tools = [SerperDevTool(), ScrapeWebsiteTool()],
)

In [46]:
# define google research task
channel_research_task = Task(
    description = f"""Research the channel {youtube_lead['name']} at {youtube_lead['channel_url']} 
    1. Identify channel focus and content type.
    2. Determine approximate subscriber count and engagement.
    3. Check for audience alignment with {brand_info['name']}.""",
    expected_output = "A research report on the channel's key attributes and audience",
    agent = research_agent
)

In [47]:
# define the research video task
video_research_task = Task(
    description= f"""
    Research the most popular videos from channel {youtube_lead['name']} at {youtube_lead['channel_url']}
    Summarise their topics and analyse the audiences""",
    expected_output= "A research report on the channel's most popular videos and their topics, including audience engagement and feedback.",
    agent = research_agent
)


In [34]:
## set youtube tool custom model and embeddings
youtube_channel_tool = YoutubeChannelSearchTool(
    config=dict(
        llm=dict(
            provider="google",
            config=dict(
                model="gemini-2.0-flash-lite",
                temperature=0.5,
            ),
        ),
        embedder=dict(
            provider="google",
            config=dict(
                model="models/text-embedding-004",
                task_type="retrieval_document",
            ),
        ),
    )
)

youtube_video_tool = YoutubeVideoSearchTool(
    config=dict(
        llm=dict(
            provider="google",
            config=dict(
                model="gemini-2.0-flash-lite",
                temperature=0.5,
            ),
        ),
        embedder=dict(
            provider="google",
            config=dict(
                model="models/text-embedding-004",
                task_type="retrieval_document",
            ),
        ),
    )
)

In [48]:
# Define a sentiment on the channel via Google searches
sentiment_task = Task(
    description = f"""
    Conduct a sentiment analysis via Google Search on the channel {youtube_lead['name']} at {youtube_lead['channel_url']}
    Look for feedbacks, reviews, or discussions that indicate general sentiment.
    Search for instance on reddit""",
    expected_output = "A sentiment analysis report on the channel's .",
    tools= [SerperDevTool(), ScrapeWebsiteTool(), youtube_channel_tool, youtube_video_tool],
    agent = research_agent
)

# Build AI Crew

In [49]:
# Assemble the AI crew

crew = Crew(
    agents= [research_agent],
    tasks=[channel_research_task,video_research_task,sentiment_task
    ],
    process = Process.sequential,
    verbose= True
)

result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5b8cdbeb-b039-4d68-b539-cc380120dee7                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 675ad6e5-9ddb-4337-836b-7c0b1f495661
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 675ad6e5-9ddb-4337-836b-7c0b1f495661
       Status: Executing Task...
    └── 🤖 Agent: Influencer researcher
            Status: In Progress

# Agent: Influencer researcher
## Task: Research the channel Theo - t3․gg at https://www.youtube.com/@t3dotgg 
    1. Identify channel focus and content type.
    2. Determine approximate subscriber count and engagement.
    3. Check for audience alignment with Zero to Mastery.


🤖 Agent: Influencer researcher
    Status: In Progress

...



# Agent: Influencer researcher
## Thought: The Reddit thread "Theo - t3.gg reviews 'Going back to Next' after giving Go a try." on r/theprimeagen is a general discussion thread. Without further investigation of the comments within the thread, it's difficult to determine the overall sentiment.
To get a better understanding of the sentiment, I will try to search for comments within the youtube channel using the search tool.
## Using tool: Search a Youtube Channels content
## Tool Input: 
"{\"search_query\": \"reviews or feedback or sentiment\", \"youtube_channel_handle\": \"@t3dotgg\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: 400 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
] in query..
 Tool Search a Youtube Channels content ac

🚀 Crew: crew
├── 📋 Task: 675ad6e5-9ddb-4337-836b-7c0b1f495661
│      Assigned to: Influencer researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Influencer researcher
│           Status: ✅ Completed
├── 📋 Task: c90238e4-fd43-4d56-bcd1-3d24d88adf1e
│      Assigned to: Influencer researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Influencer researcher
│           Status: ✅ Completed
└── 📋 Task: d58d863e-45db-423e-9010-9881b892f369
       Status: Executing Task...
    └── 🤖 Agent: Influencer researcher
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 675ad6e5-9ddb-4337-836b-7c0b1f495661
│      Assigned to: Influencer researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Influencer researcher
│           Status: ✅ Completed
├── 📋 Task: c90238e4-fd43-4d56-bcd1-3d24d88adf1e
│      Assigned to: Influencer researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Influencer researcher
│           Status: ✅ Completed
└── 📋 Task: d58d863e-45db-423e-9010-9881b892f369
       Assigned to: Influencer researcher
       Status: ✅ Completed
    └── 🤖 Agent: Influencer researcher
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d58d863e-45db-423e-9010-9881b892f369                                                                     │
│  Agent: Influencer researcher                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5b8cdbeb-b039-4d68-b539-cc380120dee7                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [51]:
# test llm if working
channelresearch_output = result.tasks_output[0].raw
video_research_output = result.tasks_output[1].raw
sentiment_output = result.tasks_output[2].raw

# Display the results
print("Channel Research Output:")
display(Markdown(channelresearch_output))
print("Video Research Output:")
display(Markdown(video_research_output))
print("Sentiment Analysis Output:")
display(Markdown(sentiment_output))

Channel Research Output:


```
**Research Report: Theo - t3․gg YouTube Channel**

**Channel Focus and Content Type:**

Theo's YouTube channel, t3․gg, focuses on software development, particularly full-stack TypeScript. The content includes tutorials, opinion pieces on tech trends, and explorations of new tools and technologies. He often discusses topics relevant to modern web development, such as AI, new frameworks, and the T3 stack (his own creation).

**Subscriber Count and Engagement:**

Based on multiple sources, Theo's channel has approximately 425,000 subscribers. His videos generally receive tens of thousands to hundreds of thousands of views, indicating a high level of engagement.

**Audience Alignment with Zero to Mastery:**

Theo's audience consists of software developers, ranging from beginners to experienced professionals. This aligns well with Zero to Mastery's target audience, which focuses on providing comprehensive education for aspiring and current developers. His focus on modern web development technologies, such as TypeScript and full-stack development, is also relevant to Zero to Mastery's course offerings.

**Overall Assessment:**

Theo's YouTube channel presents a strong potential partnership opportunity for Zero to Mastery. His audience is highly relevant, his content is engaging, and his expertise in modern web development aligns well with Zero to Mastery's educational focus. Partnering with Theo could help Zero to Mastery reach a wider audience of developers interested in learning the latest technologies and advancing their careers.
```

Video Research Output:


**Research Report: Theo - t3․gg YouTube Channel**

**Channel Focus and Content Type:**

Theo's YouTube channel, t3․gg, focuses on software development, particularly full-stack TypeScript. The content includes tutorials, opinion pieces on tech trends, and explorations of new tools and technologies. He often discusses topics relevant to modern web development, such as AI, new frameworks, and the T3 stack (his own creation).

Based on the search results, recent popular videos include:

*   "PirateSoftware is right, this needs to stop" (936K views - 1 month ago, according to a previous search result)
*   "The REAL Cost Of AWS (And How To Avoid It)" (525K views - 1 year ago, according to a previous search result)
*   "Figma threatens companies using "Dev Mode"" (98K views)
*   "Don't sleep on Chef (I can't believe it works this well)" (98K views)
*   "The most important function in my codebase" (115K views)

These titles suggest a mix of content, including commentary on industry news ("Figma threatens companies using 'Dev Mode'"), practical advice ("The REAL Cost Of AWS"), and explorations of specific technologies ("Don't sleep on Chef").

**Subscriber Count and Engagement:**

Based on multiple sources, Theo's channel has approximately 425,000 subscribers. His videos generally receive tens of thousands to hundreds of thousands of views, indicating a high level of engagement.

**Audience Alignment with Zero to Mastery:**

Theo's audience consists of software developers, ranging from beginners to experienced professionals. This aligns well with Zero to Mastery's target audience, which focuses on providing comprehensive education for aspiring and current developers. His focus on modern web development technologies, such as TypeScript and full-stack development, is also relevant to Zero to Mastery's course offerings.

**Audience Engagement and Feedback (Inferred):**

While direct audience feedback is not accessible with the current tools, the high view counts on Theo's videos suggest strong audience interest. The topics covered, such as AWS costs and Figma's Dev Mode, are likely to generate discussion and engagement within the developer community. His focus on practical advice and modern technologies probably resonates with developers seeking to improve their skills and stay up-to-date with industry trends.

**Overall Assessment:**

Theo's YouTube channel presents a strong potential partnership opportunity for Zero to Mastery. His audience is highly relevant, his content is engaging, and his expertise in modern web development aligns well with Zero to Mastery's educational focus. Partnering with Theo could help Zero to Mastery reach a wider audience of developers interested in learning the latest technologies and advancing their careers.

Sentiment Analysis Output:


Based on the available information, the sentiment towards Theo - t3․gg's YouTube channel appears to be mixed. While his technical expertise and content quality are generally well-regarded, some viewers express concerns about potential bias due to sponsorships and paid promotions.

Here's a breakdown:

*   **Positive Sentiment:** Theo is seen as knowledgeable and up-to-date on modern web development technologies. His tutorials and explorations of new tools are likely appreciated by developers seeking to improve their skills. The high view counts on his videos suggest a strong level of audience interest and engagement.
*   **Neutral Sentiment:** Many viewers likely appreciate the content without strong positive or negative feelings.
*   **Negative Sentiment:** Some viewers express skepticism about his objectivity due to potential sponsorships or paid promotions. They perceive a "vibe of paid promotion" in his videos, which makes them question the authenticity of his opinions. This doesn't necessarily mean viewers believe he is lying, but it creates a sense of distrust.

It's important to note that this analysis is based on limited information and may not represent the full spectrum of opinions about Theo's channel. A more comprehensive analysis would require access to more data, such as YouTube comments, Reddit discussions, and social media mentions.